## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# pd.options.mode.use_inf_as_na = True

In [32]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Kyzyl,RU,51.7000,94.4500,60.03,75,87,2.93,overcast clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,70.56,87,45,7.00,scattered clouds
2,2,Lianhe,CN,47.1333,129.2667,56.44,87,100,3.18,overcast clouds
3,3,Okhotsk,RU,59.3833,143.3000,64.11,69,24,7.83,few clouds
4,4,Barra Velha,BR,-26.6322,-48.6847,64.31,96,1,3.40,clear sky


In [33]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
8,8,Sabzawār,IR,36.2126,57.6819,81.39,18,0,6.69,clear sky
10,10,Albany,US,42.6001,-73.9662,83.05,65,22,5.88,few clouds
14,14,Keti Bandar,PK,24.1447,67.4497,79.68,76,96,13.11,overcast clouds
31,31,Atuona,PF,-9.8000,-139.0333,77.77,79,23,21.03,light rain
32,32,Puerto Carreño,CO,6.1890,-67.4859,79.59,79,66,6.60,broken clouds


In [35]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        243
City           243
Country        243
Lat            243
Lng            243
Max Temp       243
Humidity       243
Cloudiness     243
Wind Speed     243
Description    243
dtype: int64

In [36]:
city_data_df.count()

City_ID        708
City           708
Country        706
Lat            708
Lng            708
Max Temp       708
Humidity       708
Cloudiness     708
Wind Speed     708
Description    708
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df = city_data_df.dropna()
city_data_df.count()

City_ID        706
City           706
Country        706
Lat            706
Lng            706
Max Temp       706
Humidity       706
Cloudiness     706
Wind Speed     706
Description    706
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(3)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Sabzawār,IR,81.39,clear sky,36.2126,57.6819,
10,Albany,US,83.05,few clouds,42.6001,-73.9662,
14,Keti Bandar,PK,79.68,overcast clouds,24.1447,67.4497,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    
    # 6c.Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     hotel_df.loc[index, "Hotel Name"] = "No Hotel"
     print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Sabzawār,IR,81.39,clear sky,36.2126,57.6819,پزشکی قانونی سبزوار
10,Albany,US,83.05,few clouds,42.6001,-73.9662,No Hotel
14,Keti Bandar,PK,79.68,overcast clouds,24.1447,67.4497,Haji laloo memon house
31,Atuona,PF,77.77,light rain,-9.8000,-139.0333,Villa Enata
32,Puerto Carreño,CO,79.59,broken clouds,6.1890,-67.4859,Casa Hotel Victoria


In [41]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.drop(index=hotel_df[hotel_df['Hotel Name'] == 'No Hotel'].index, inplace=True)

hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
8,Sabzawār,IR,81.39,clear sky,36.2126,57.6819,پزشکی قانونی سبزوار
14,Keti Bandar,PK,79.68,overcast clouds,24.1447,67.4497,Haji laloo memon house
31,Atuona,PF,77.77,light rain,-9.8000,-139.0333,Villa Enata
32,Puerto Carreño,CO,79.59,broken clouds,6.1890,-67.4859,Casa Hotel Victoria
38,Butaritari,KI,83.73,broken clouds,3.0707,172.7902,Isles Sunset Lodge


In [42]:
hotel_df.count()

City           223
Country        223
Max Temp       223
Description    223
Lat            223
Lng            223
Hotel Name     223
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

print (f"Showing results for minimum temperature: {min_temp} and maximum temperature: {max_temp}")

fig


Showing results for minimum temperature: 75.0 and maximum temperature: 90.0


Figure(layout=FigureLayout(height='420px'))